In [55]:
import quandl, math
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import datetime
import matplotlib as plt
from matplotlib import style 

df = quandl.get("WIKI/GOOGL")

print(df.head())
#print(df.tail())

df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]

df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

forecast_col = 'Adj. Close'
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))

df['label'] = df[forecast_col].shift(-forecast_out)

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [31]:
df.dropna(inplace=True)
df.drop(['label'],1)

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.072956,0.324968,44659000.0
2004-08-20,54.322689,7.921706,7.227007,22834300.0
2004-08-23,54.869377,4.049360,-1.227880,18256100.0
2004-08-24,52.597363,7.657099,-5.726357,15247300.0
2004-08-25,53.164113,3.886792,1.183658,9188600.0
2004-08-26,54.122070,3.048837,2.820391,7094800.0
2004-08-27,53.239345,2.760245,-1.803885,6211700.0
2004-08-30,51.162935,3.411430,-3.106003,5196700.0
2004-08-31,51.343492,1.514115,0.048866,4917800.0


In [32]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])

In [38]:
X = preprocessing.scale(X)

X=X[:-forecast_out]
X_lately = X[-forecast_out:]

In [34]:
#X = X[:-forecast_out+1]

In [35]:
df.dropna(inplace=True)

In [36]:
y = np.array(df['label'])

In [37]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size=0.2)


ValueError: Found input variables with inconsistent numbers of samples: [3269, 3303]

In [25]:
#clf = LinearRegression()
#clf = svm.SVR(kernel='poly')
clf = LinearRegression(n_jobs=10)

In [26]:
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=False)

In [27]:
accuray = clf.score(X_test, y_test)

In [28]:
accuray

0.97663884437646398

In [39]:
forecast_Set = clf.predict(X_lately)

In [41]:
print(forecast_Set, accuray, forecast_out)

[  996.04399441   994.05804824   992.54861034   993.63041836   992.89670104
   996.88271771  1002.93675351   976.50109792   987.51595772   992.35729337
  1002.24680257  1008.34427521  1016.27839389  1029.80771419  1031.92978705
  1035.40632922  1023.95470161  1026.35979742  1035.08151519  1042.82409995
  1034.54235331  1040.78452688  1043.23819149  1000.35540464   995.94331876
  1007.31227096  1003.99007249   996.62546562   995.58139676  1013.28598719
  1006.16719395  1017.03691034  1015.10636618  1024.61457151] 0.976638844376 34


In [53]:
forecast_set = clf.predict(X_lately)
df['forcast'] = np.nan

In [44]:
last_date = df.iloc[-1].name

In [46]:
last_unix = last_date.timestamp()

In [47]:
one_day = 86400

In [51]:
next_unix = last_unix + one_day

In [60]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += 86400
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

df['Adj. Close'].plot()
df['Forecast'].plot()
# #plt.legend(loc=4)
# plt.xlabel('Date')
# plt.ylabel('Price')
plt.show()

KeyError: 'Forecast'